In [49]:
from flask import Flask
from flask import request
import werkzeug
import cv2
from PIL import Image
import mediapipe as mp
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import threading

In [50]:
class Results:
    #TRAGULAR DATA HAS SHAP(EAR COORD,INDEX COORD)
    trag_cols = ["x","y","z","vis"]
    trag_left_rows = ["left_ear","left_index"]
    trag_right_rows = ["right_ear","right_index"]
    tragular_left_coord = []
    tragular_right_coord = []

In [51]:
def mp_pose_tragular(side,file):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    BG_COLOR = (192, 192, 192) # gray
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True,model_complexity=2,enable_segmentation=True,min_detection_confidence=0.5)
    image = cv2.imread(file)
    scale_percent = 220 # percent of original size
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    annotated_image = image.copy()
    # Draw pose landmarks on the image.
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    cv2.imwrite('annotated_image' + '.jpg', annotated_image)
    # Plot pose world landmarks.
    mp_drawing.plot_landmarks(
        results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    
    if(side == 0):
        coordinate_list = [[results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].z,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].visibility],
                                       [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].x,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].y,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].z,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].visibility]]
        Results.tragular_left_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_left_coord.index = Results.trag_left_rows
        print(Results.tragular_left_coord)
    else:
        coordinate_list = [[results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].x,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].y,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].z,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].visibility],
                                       [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].x,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].y,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].z,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].visibility]]
        Results.tragular_right_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_right_coord.index = Results.trag_right_rows
        print(Results.tragular_right_coord)
    

In [ ]:
app = Flask(__name__)

@app.route("/")
def showHomePage():
    return "This is home page"

@app.route("/debug",methods=["POST"])
def debug():
    text = request.form["sample"]
    print(text)
    return "received"

@app.route("/tragularLeft",methods=["POST"])
def tragularLeft():
    imageFile = request.files['leftTragular']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_tragular(0,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/tragularRight",methods=["POST"])
def tragularRight():
    imageFile = request.files['rightTragular']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_tragular(1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"
    

if __name__ == "__main__":
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://138.38.166.67:5000/ (Press CTRL+C to quit)



Received image file name: right_tragular.jpg


/Users/louisgodtfredsen/opt/anaconda3/lib/python3.9/site-packages/mediapipe/python/solutions/drawing_utils.py:318: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
138.38.214.43 - - [12/Jul/2022 18:49:55] "POST /tragularRight HTTP/1.1" 200 -


                    x         y         z       vis
right_ear    0.547792  0.422371 -0.413779  0.999962
right_index  0.490231  0.480494 -1.062891  0.990235

Successful upload

Received image file name: left_tragular.jpg


/Users/louisgodtfredsen/opt/anaconda3/lib/python3.9/site-packages/mediapipe/python/solutions/drawing_utils.py:318: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
138.38.214.43 - - [12/Jul/2022 18:51:15] "POST /tragularLeft HTTP/1.1" 200 -


                   x         y         z       vis
left_ear    0.577219  0.187456 -0.555628  0.999987
left_index  0.865005  0.664403 -0.639623  0.991629

Successful upload
